## Introduction

So far, it has been observed that prompt engineering using few shot examples from the LegalRuleML core specification (v1.0) can generate an XML that is satisfactory at least as a starting point. However, one of the problems faced in this process is that of non-observance of some of the rules of the schema. Using the metamodel provided by the LegalRuleML provides a hint as to what an ideal representation should be.

## Table of Contents
[Approach 1](#Approach-1) - Using a Basic RAG approach on the MetaModel
[Approach 2](#Approach-2) - Adapting the prompt with an explanation on the metamodel topic.

## Results/Conclusion
It is observed that providing the model a bigger token size, as well as information about the metamodel gives a better output. This means incorporating the XML similarity 

# Setup

In [4]:
from langchain.schema import HumanMessage
from langchain_community.llms.azureml_endpoint import AzureMLEndpointApiType
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint
from langchain_community.chat_models.azureml_endpoint import LlamaChatContentFormatter

from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [6]:
llm = AzureMLChatOnlineEndpoint(
    endpoint_url="",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key="",
    content_formatter=LlamaChatContentFormatter(),
    model_kwargs={"temperature": 0.4, "max_tokens": 1000}
)
response = llm.invoke(
    [HumanMessage(content="Will the Collatz conjecture ever be solved?")]
)
response

BaseMessage(content='The Collatz Conjecture is a longstanding open problem in mathematics, and it is difficult to predict with certainty whether it will be solved in the future. However, the conjecture has been extensively studied and many attempts have been made to prove it, but so far, no one has been able to find a rigorous proof.\n\nThe Collatz Conjecture is a simple-sounding problem: given a positive integer, repeatedly apply the simple transformation: if the number is even, divide it by 2, and if it is odd, multiply it by 3 and add 1. The conjecture states that no matter what positive integer you start with, this process will always converge to the number 1.\n\nDespite the simplicity of the conjecture, it has proven to be extraordinarily difficult to prove or disprove. Many mathematicians have attempted to solve it, but so far, no one has been able to find a rigorous proof or counterexample. In fact, the Collatz Conjecture is one of the most famous unsolved problems in mathematic

In [26]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List


class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        page_content = """

        <rdf:RDF xml:base="http://docs.oasis-open.org/legalruleml/ns/v1.0/metamodel">
<!--
<owl:Ontology rdf:about="&lrmlmm;/modules/defeasible.rdfs" dc:title="The LegalRuleML MetaModel Defeasible Rules vocabulary"/>
-->
<rdfs:Class rdf:about="#DefeasibleStrength">
<rdfs:label>defeasible strength</rdfs:label>
<rdfs:comment>
an indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the conclusion of the Legal Rule holds.
</rdfs:comment>
<rdfs:subClassOf rdf:resource="#Strength"/>
</rdfs:Class>
<rdfs:Class rdf:about="#Defeater">
<rdfs:label>defeater</rdfs:label>
<rdfs:comment>
an indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the opposite of the conclusion of the Legal Rule does not hold.
</rdfs:comment>
<rdfs:subClassOf rdf:resource="#Strength"/>
</rdfs:Class>
<rdfs:Class rdf:about="#Override">
<rdfs:label>override</rdfs:label>
<rdfs:comment>
an indication that a Legal Rule takes precedence over another Legal Rule. The ordered pair of Legal Rules is an instance in a defeasible priority relation.
</rdfs:comment>
<rdfs:subClassOf rdf:resource="http://docs.oasis-open.org/legalruleml/ns/v1.0/rule-metamodel#Atom"/>
</rdfs:Class>
<!-- Import from RuleML for diagramming purposes -->
<rdfs:Class rdf:about="http://docs.oasis-open.org/legalruleml/ns/v1.0/rule-metamodel#Atom">
<rdfs:subClassOf rdf:resource="http://docs.oasis-open.org/legalruleml/ns/v1.0/rule-metamodel#Formula"/>
</rdfs:Class>
<rdfs:Class rdf:about="#Strength">
<rdfs:label>strength</rdfs:label>
<rdfs:comment>
the quality of a Legal Rule to resist or not to resist a rebuttal.
</rdfs:comment>
<rdfs:subClassOf rdf:resource="#Associable"/>
</rdfs:Class>
<rdfs:Class rdf:about="#StrictStrength">
<rdfs:label>strict strength</rdfs:label>
<rdfs:comment>
an indication that where the antecedent of a Legal Rule is indisputable, the conclusion of the Legal Rule is indisputable.
</rdfs:comment>
<rdfs:subClassOf rdf:resource="#Strength"/>
</rdfs:Class>
<rdf:Property rdf:about="#appliesStrength">
<rdfs:label>applies strength</rdfs:label>
<rdfs:comment>a Strength applied by the Context or Association.</rdfs:comment>
<rdfs:range rdf:resource="#Strength"/>
<rdfs:subPropertyOf rdf:resource="#applies"/>
</rdf:Property>
<rdf:Property rdf:about="#hasQualification">
<rdfs:label>has qualifications</rdfs:label>
<rdfs:comment>
a qualification (e.g. an Override) of the Statements.
</rdfs:comment>
<rdfs:domain rdf:resource="#Statements"/>
<rdfs:range rdf:resource="http://docs.oasis-open.org/legalruleml/ns/v1.0/rule-metamodel#Formula"/>
</rdf:Property>
<rdf:Property rdf:about="#hasStrength">
<rdfs:label>hasStrength</rdfs:label>
<rdfs:comment>the Strength of the Legal Rule.</rdfs:comment>
<rdfs:domain rdf:resource="#Statement"/>
<rdfs:range rdf:resource="#Strength"/>
</rdf:Property>
<rdf:Property rdf:about="#over">
<rdfs:label>over</rdfs:label>
<rdfs:comment>the Legal Rule with higher priority.</rdfs:comment>
<rdfs:domain rdf:resource="#Override"/>
<rdfs:range rdf:resource="#Statement"/>
</rdf:Property>
<rdf:Property rdf:about="#under">
<rdfs:label>under</rdfs:label>
<rdfs:comment>the Legal Rule with lower priority.</rdfs:comment>
<rdfs:domain rdf:resource="#Override"/>
<rdfs:range rdf:resource="#Statement"/>
</rdf:Property>
<!--
Duplicate from context.rdfs for diagramming purposes 
-->
<rdf:Property rdf:about="#applies">
<rdfs:domain rdf:resource="#Associator"/>
<rdfs:range rdf:resource="#Associable"/>
</rdf:Property>
</rdf:RDF>
"""
        
        return [Document(page_content=page_content)]

retrieverObj = CustomRetriever()

retriever = retrieverObj.get_relevant_documents("query")

print(retriever)

[Document(page_content='\n\n        <rdf:RDF xml:base="http://docs.oasis-open.org/legalruleml/ns/v1.0/metamodel">\n<!--\n<owl:Ontology rdf:about="&lrmlmm;/modules/defeasible.rdfs" dc:title="The LegalRuleML MetaModel Defeasible Rules vocabulary"/>\n-->\n<rdfs:Class rdf:about="#DefeasibleStrength">\n<rdfs:label>defeasible strength</rdfs:label>\n<rdfs:comment>\nan indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the conclusion of the Legal Rule holds.\n</rdfs:comment>\n<rdfs:subClassOf rdf:resource="#Strength"/>\n</rdfs:Class>\n<rdfs:Class rdf:about="#Defeater">\n<rdfs:label>defeater</rdfs:label>\n<rdfs:comment>\nan indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the opposite of the conclusion of the Legal Rule does not hold.\n</rdfs:comment>\n<rdfs:subClassOf rdf:resource="#Strength"/>\n</rdfs:Class>\n<rdfs:Class rdf:about="#Override">\n<rdfs:label>override</rdfs:l

In [15]:
question = """Align this code with the metamodel provided to you in rsfd format
<lrml:LegalSources>
  <lrml:LegalSource key="ls1" sameAs="http://www.example.com/rules/rule5"/>
</lrml:LegalSources>

<lrml:Context key="psInfo1">
  <lrml:appliesAssociations>
    <lrml:Associations>
      <lrml:Association keyref="#ls1"/>
    </lrml:Associations>
  </lrml:appliesAssociations>
</lrml:Context>

<lrml:Statements key="textblock1">
  <lrml:hasQualification>
    <lrml:Override over="#psInfo1" under="#psInfo2"/>
  </lrml:hasQualification>
  <lrml:PrescriptiveStatement key="ps1">
    <ruleml:Rule key=":rule5" closure="universal">
      <lrml:if>
        <lrml:Atom>
          <lrml:Rel iri=":operationWithoutExtendedProducerResponsibilityAuthorization"/>
          <lrml:Var>X</lrml:Var>
        </lrml:Atom>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <ruleml:Atom>
            <lrml:Rel iri=":causeEnvironmentalDamage"/>
            <lrml:Var>X</lrml:Var>
          </ruleml:Atom>
        </lrml:Obligation>
      </lrml:then>
    </ruleml:Rule>
  </lrml:PrescriptiveStatement>
</lrml:Statements>
"""

context = ""

prompt_content = f"""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:"""

# Note: Replace {context_placeholder} with the actual context from the RAG pipeline when executing the prompt.


In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Assuming retriever is an object of CustomRetriever class
retriever = CustomRetriever()

prompt_content

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_content
    | llm
    | StrOutputParser()
)


TypeError: unsupported operand type(s) for |: 'dict' and 'str'

## Approach-1
Using a Basic RAG approach on the MetaModel


In [29]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt_template= """
### [INST] 
You are an assistant for making the XML generated in accordance with the rdfs metamodel provided to you below as context. 
Use the following context to ensure that the XML generated is in terms of the metamodel specification.



{context}

### QUESTION:
{question} 

[/INST]
"""
 
# Abstraction of Prompt
prompt = ChatPromptTemplate.from_template(prompt_template)
output_parser = StrOutputParser()

# Creating an LLM Chain 
llm_chain = LLMChain(llm=llm, prompt=prompt)

# RAG Chain
rag_chain = ( 
 {"context": retrieverObj, "question": RunnablePassthrough()}
    | llm_chain
)

In [30]:
rag_chain.invoke("""
<lrml:LegalSources>
  <lrml:LegalSource key="ls1" sameAs="http://www.example.com/rules/rule5"/>
</lrml:LegalSources>

<lrml:Context key="psInfo1">
  <lrml:appliesAssociations>
    <lrml:Associations>
      <lrml:Association keyref="#ls1"/>
    </lrml:Associations>
  </lrml:appliesAssociations>
</lrml:Context>

<lrml:Statements key="textblock1">
  <lrml:hasQualification>
    <lrml:Override over="#psInfo1" under="#psInfo2"/>
  </lrml:hasQualification>
  <lrml:PrescriptiveStatement key="ps1">
    <ruleml:Rule key=":rule5" closure="universal">
      <lrml:if>
        <lrml:Atom>
          <lrml:Rel iri=":operationWithoutExtendedProducerResponsibilityAuthorization"/>
          <lrml:Var>X</lrml:Var>
        </lrml:Atom>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <ruleml:Atom>
            <lrml:Rel iri=":causeEnvironmentalDamage"/>
            <lrml:Var>X</lrml:Var>
          </ruleml:Atom>
        </lrml:Obligation>
      </lrml:then>
    </ruleml:Rule>
  </lrml:PrescriptiveStatement>
</lrml:Statements>""")

{'context': [Document(page_content='\n\n        <rdf:RDF xml:base="http://docs.oasis-open.org/legalruleml/ns/v1.0/metamodel">\n<!--\n<owl:Ontology rdf:about="&lrmlmm;/modules/defeasible.rdfs" dc:title="The LegalRuleML MetaModel Defeasible Rules vocabulary"/>\n-->\n<rdfs:Class rdf:about="#DefeasibleStrength">\n<rdfs:label>defeasible strength</rdfs:label>\n<rdfs:comment>\nan indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the conclusion of the Legal Rule holds.\n</rdfs:comment>\n<rdfs:subClassOf rdf:resource="#Strength"/>\n</rdfs:Class>\n<rdfs:Class rdf:about="#Defeater">\n<rdfs:label>defeater</rdfs:label>\n<rdfs:comment>\nan indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the opposite of the conclusion of the Legal Rule does not hold.\n</rdfs:comment>\n<rdfs:subClassOf rdf:resource="#Strength"/>\n</rdfs:Class>\n<rdfs:Class rdf:about="#Override">\n<rdfs:label>over

# Approach-2 
Adapting the prompt with an explanation on the metamodel topic.


In [33]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt_template= """
### [INST] 
<<SYS>> You are an assistant that takes the XML generated and tweaks it in accordance with the rdfs metamodel provided to you below as context. 
The XML is in compliance with the LegalRuleML core specification, whereas the metamodel provides an overview of what the XML should be for a particular category of XML elements.
The category being implemented is "defeasibility" which is defined as follows:

'Defeasibility is the property that a conclusion is open in principle to revision in case more evidence to the contrary is provided. Defeasible reasoning is in contrast to monotonic reasoning of propositional logic, where no revision is possible. In addition, defeasible reasoning allows reasoning in the face of contradictions, which gives rise to ex false quodlibet in propositional logic. One application of defeasible reasoning is the ability to model exceptions in a simple and natural way.'
<</SYS>>
Use the following metamodel to ensure that the XML generated is in terms of the definition of defeasibility provided in the LegalRuleML specification.

{context}

### QUESTION:
{question} 

[/INST]
"""
 
# Abstraction of Prompt
prompt = ChatPromptTemplate.from_template(prompt_template)
output_parser = StrOutputParser()

# Creating an LLM Chain 
llm_chain = LLMChain(llm=llm, prompt=prompt)

# RAG Chain
rag_chain = ( 
 {"context": retrieverObj, "question": RunnablePassthrough()}
    | llm_chain
)

In [34]:
rag_chain.invoke("""
<lrml:LegalSources>
  <lrml:LegalSource key="ls1" sameAs="http://www.example.com/rules/rule5"/>
</lrml:LegalSources>

<lrml:Context key="psInfo1">
  <lrml:appliesAssociations>
    <lrml:Associations>
      <lrml:Association keyref="#ls1"/>
    </lrml:Associations>
  </lrml:appliesAssociations>
</lrml:Context>

<lrml:Statements key="textblock1">
  <lrml:hasQualification>
    <lrml:Override over="#psInfo1" under="#psInfo2"/>
  </lrml:hasQualification>
  <lrml:PrescriptiveStatement key="ps1">
    <ruleml:Rule key=":rule5" closure="universal">
      <lrml:if>
        <lrml:Atom>
          <lrml:Rel iri=":operationWithoutExtendedProducerResponsibilityAuthorization"/>
          <lrml:Var>X</lrml:Var>
        </lrml:Atom>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <ruleml:Atom>
            <lrml:Rel iri=":causeEnvironmentalDamage"/>
            <lrml:Var>X</lrml:Var>
          </ruleml:Atom>
        </lrml:Obligation>
      </lrml:then>
    </ruleml:Rule>
  </lrml:PrescriptiveStatement>
</lrml:Statements>""")

{'context': [Document(page_content='\n\n        <rdf:RDF xml:base="http://docs.oasis-open.org/legalruleml/ns/v1.0/metamodel">\n<!--\n<owl:Ontology rdf:about="&lrmlmm;/modules/defeasible.rdfs" dc:title="The LegalRuleML MetaModel Defeasible Rules vocabulary"/>\n-->\n<rdfs:Class rdf:about="#DefeasibleStrength">\n<rdfs:label>defeasible strength</rdfs:label>\n<rdfs:comment>\nan indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the conclusion of the Legal Rule holds.\n</rdfs:comment>\n<rdfs:subClassOf rdf:resource="#Strength"/>\n</rdfs:Class>\n<rdfs:Class rdf:about="#Defeater">\n<rdfs:label>defeater</rdfs:label>\n<rdfs:comment>\nan indication that, in the absence of information to the contrary and where the antecedent of a Legal Rule holds, the opposite of the conclusion of the Legal Rule does not hold.\n</rdfs:comment>\n<rdfs:subClassOf rdf:resource="#Strength"/>\n</rdfs:Class>\n<rdfs:Class rdf:about="#Override">\n<rdfs:label>over